In [1]:
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os 
import numpy as np
import scipy as sci
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as pl

import scanpy as sc
import scirpy as ir
import anndata as ann

from scipy.sparse import csr_matrix
from matplotlib import rcParams
from matplotlib import colors

sc.settings.verbosity = 3


In [2]:
sc.settings.set_figure_params(dpi=70)
sc.settings.verbosity = 3
sc.settings.n_jobs = 3
sc.set_figure_params(vector_friendly=True,color_map='viridis',dpi_save=600,transparent=True)

Paths to preprocessed data at different stages.

In [3]:
path_base = '/ix1/ylee/Yifan_Zhang/CellRanger_tests'
path_merged = path_base + 'merged_tcr_10x.h5ad'
path_filtered = path_base + 'filtered_tcr_10x.h5ad'
path_logged = path_base + 'logged_10x.h5ad'
path_highlyvar = path_base + 'highly_var_5000.h5ad'
path_clean_split = path_base + 'v5_train_val_test.h5ad'
path_supervised = path_base + 'v6_supervised.h5ad'
path_avidity = path_base + 'v7_avidity.h5ad'

## Merge Files

The 10x VDJ dataset consists of CD8+ T cells from for donor. First, transcriptome data is merged with tcr information. In a second step, the data is fused over all donors. In the third step, the measured binding affinity is merged.

In [14]:
import muon as mu
mdata = mu.read("/ihome/ylee/yiz133/Code/Data processing/all_Common_DEtop2500.h5mu")

adata = mdata['gex']

In [15]:
adata

AnnData object with n_obs × n_vars = 77502 × 2500
    obs: 'mouse_id', 'date', 'tissue', 'sample', 'mouse_BC', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'n_genes', 'n_counts', 'leiden', 'modified_cell_type', 'IFN_stim', 'Activation', 'Exhaust', 'Mem', 'state', 'is_2D2'
    var: 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'hvg', 'is_2D2_colors', 'leiden', 'leiden_colors', 'log1p', 'modified_cell_type_col

In [16]:
count_irs = sum([1 for x in adata.obs['has_ir'] if x=='True'])
print(f'Total Count: {len(adata.obs)}')
print(f'With IR-Info: {count_irs}')

KeyError: 'has_ir'

In [ ]:
# 237883
adata.obs['donor'].value_counts()

# Qualitiy control

Load data and calculate the amount of counts, genes and fraction of mitochondrial genes.

In [ ]:
adata = sc.read(path_merged)
adata.obs['n_counts'] = adata.X.sum(1)
adata.obs['log_counts'] = np.log10(adata.obs['n_counts'])

adata.obs['n_genes'] = (adata.X > 0).sum(1)

mt_gene_mask = [gene.startswith('MT-') for gene in adata.var_names]
mt_gene_index = np.where(mt_gene_mask)[0]
adata.obs['mt_fraction'] = adata.X[:, mt_gene_index].sum(1) / adata.X.sum(1)

In [ ]:
rcParams['figure.figsize'] = (15, 5)
sc.pl.violin(adata, ['log_counts'], groupby='donor', size=1, log=False, rotation=90)
sc.pl.violin(adata, ['n_genes'], groupby='donor', size=1, log=False, rotation=90)
sc.pl.violin(adata, ['mt_fraction'], groupby='donor', size=1, log=False, rotation=90)

In [ ]:
rcParams['figure.figsize'] = (8, 8)
sc.pl.scatter(adata, y='n_genes', x='n_counts', color='mt_fraction', size=10, show=False)

In [ ]:
sc.pl.scatter(adata[np.logical_and(adata.obs['n_genes']<2000, adata.obs['n_counts']<10000)],
             y='n_genes', x='n_counts', color='mt_fraction', size=10, show=False)
sc.pl.scatter(adata[np.logical_and(adata.obs['n_genes']<1000, adata.obs['n_counts']<10000)],
             y='n_genes', x='n_counts', color='donor', size=10, show=False)

## Cell Filtering

Filter cells by the following indicators:
- fraction of mitochondrial genes (max)
- gene count (min, max)
- gene count (max)

Filter genes by:
- amount of cells representing this gene (min)


Mitochondrial filter: The fraction of mt genes to all genes is high for cell with membrane leakage.

In [ ]:
adata_tmp = adata.copy()
print(f'Total number of cells: {adata_tmp.n_obs}')
high_mt = adata_tmp.obs['mt_fraction'] < 0.2  #todo why this value
adata_tmp = adata_tmp[high_mt]
print(f'Number of cells after MT filter: {adata_tmp.n_obs}')

Gene-Count Filter between 1000 and 10.000

In [ ]:
low_count = adata_tmp.obs['n_counts'] > 1000
adata_tmp = adata_tmp[low_count]
print(f'Number of cells after minimum count filter: {adata_tmp.n_obs}')

sc.pp.filter_cells(adata_tmp, max_counts=10000)
print(f'Number of cells after maximum filter: {adata_tmp.n_obs}')

Gene-Count Filter: filter out cells with low gene count < 500

In [ ]:
low_genes = adata_tmp.obs['n_genes'] > 500
adata_tmp = adata_tmp[low_genes]
print(f'Number of cells after minimum gene filter: {adata_tmp.n_obs}')

Gene-Filter: filter out genes that are represented in less than 10 cells

In [ ]:
sc.pp.filter_genes(adata_tmp, min_cells=10)
print(f'Number of cells after gene filter: {adata_tmp.n_vars}')

## Doublet detection

In droplet based methods multiple cells might be present in a drop. The droplet score is evaluated by Scrublet.

In [ ]:
import scrublet as scr
import scipy.io
import time

In [ ]:
adata_tmp.obs['doublet_score'] = np.zeros(adata_tmp.shape[0])
adata_tmp.obs['doublet'] = np.zeros(adata_tmp.shape[0])

In [ ]:
# filtering / preprocessing parameters:
min_counts = 2
min_cells = 3
vscore_percentile = 85
n_pc = 50

# doublet detector parameters
expected_doublet_rate = 0.02
sim_doublet_ratio = 3
n_neighbors = 15

for batch in enumerate(adata_tmp.obs['donor'].cat.categories):
    print(batch)
    t0 = time.time()
    idx = np.flatnonzero(adata_tmp.obs['donor']==batch[1])
    scrub = scr.Scrublet(counts_matrix = adata_tmp[idx, :].X,
                        n_neighbors = n_neighbors,
                        sim_doublet_ratio = sim_doublet_ratio,
                        expected_doublet_rate = expected_doublet_rate)
    doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts = min_counts,
                                                              min_cells = min_cells,
                                                              n_prin_comps = n_pc,
                                                              use_approx_neighbors = True,
                                                              get_doublet_neighbor_parents = False)
    adata_tmp.obs['doublet_score'].iloc[idx] = doublet_scores
    adata_tmp.obs['doublet'].iloc[idx] = predicted_doublets
    t1 = time.time()
    print('Elapsed time: {:.1f} seconds'.format(t1-t0))

In [ ]:
rcParams['figure.figsize'] = (6, 6)
sb.distplot(adata_tmp.obs['doublet_score'], bins=100, kde=False)

In [ ]:
sc.pl.violin(adata_tmp, ['doublet_score'], groupby='donor', size=1, log=False, rotation=90)

In [ ]:
thr = 0.05
idx_filter = adata_tmp.obs['doublet_score'] <= thr

adata_tmp = adata_tmp[idx_filter].copy()
print(f'Number of cells after double filter: {adata_tmp.n_obs}')

## QC after filtering

In [ ]:
rcParams['figure.figsize'] = (15, 5)
sc.pl.violin(adata_tmp, ['log_counts'], groupby='donor', size=1, log=False, rotation=90)
sc.pl.violin(adata_tmp, ['n_genes'], groupby='donor', size=1, log=False, rotation=90)
sc.pl.violin(adata_tmp, ['mt_fraction'], groupby='donor', size=1, log=False, rotation=90)

In [ ]:
df = adata_tmp.obs[['n_genes', 'n_counts', 'donor']]
df_all = pd.DataFrame(df.groupby(by='donor')['n_genes'].apply(np.mean).values, 
                      index=df.groupby(by='donor')['n_genes'].apply(np.mean).index, 
                      columns=['mean_genes'])

df_all['median_genes'] = df.groupby(by='donor')['n_genes'].apply(np.median).values
df_all['mean_counts'] = df.groupby(by='donor')['n_counts'].apply(np.mean).values
df_all['median_counts'] = df.groupby(by='donor')['n_counts'].apply(np.median).values
df_all

In [ ]:
adata_tmp.obs['doublet'] = adata_tmp.obs['doublet'].astype(str)

In [ ]:
sc.write(adata=adata_tmp, filename=path_filtered)

## Simple normalization


In [ ]:
adata_tmp = sc.read(path_filtered)

In [ ]:
sc.pp.normalize_total(adata_tmp, target_sum=1e4)  # target sum? 
sc.pp.log1p(adata_tmp)

In [ ]:
sc.write(adata=adata_tmp, filename=path_logged)

## Highly variable Gens

In [ ]:
adata_tmp = sc.read(path_logged)

In [ ]:
adata_tmp.uns['log1p'] = {'base': None}
sc.pp.highly_variable_genes(adata_tmp, n_top_genes=5000, batch_key='donor')
print('Shape before: ', adata_tmp.shape)
adata_tmp = adata_tmp[:, adata_tmp.var['highly_variable']]
print('Shape after: ', adata_tmp.shape)

In [ ]:
sc.write(adata=adata_tmp, filename=path_highlyvar)

## Model Specific Preproccessing


### Further TC cleaning

Clean cells without or multiple IR information. Filter out chains without paired TRA and TRB (None).

In [ ]:
adata = sc.read_h5ad(path_highlyvar)

In [ ]:
print('All cells: ', adata.shape)
adata = adata[adata.obs['has_ir'] == 'True']
print('With IRs: ', adata.shape)
adata = adata[adata.obs['multi_chain'] == 'False']
print('With 1 IR set: ', adata.shape)
adata = adata[(adata.obs['IR_VJ_1_junction_aa'] != 'None') & (~adata.obs['IR_VJ_1_junction_aa'].isna()) & 
              (adata.obs['IR_VDJ_1_junction_aa'] != 'None') & (~adata.obs['IR_VDJ_1_junction_aa'].isna())]
print('With full IR:', adata.shape)

### Clonotype annotation

The clonotype annotation from 10x defines clonotypes not accross all donors. Therefor, public TCRs might leak between training and testing data. Further, cells with the same TCR but multiple IRs are classified as different clonotypes. We therefor assign own clonotype labels based on complete correspondence based on alpha and beta chain.

In [ ]:
adata.obs['TRA+TRB'] = adata.obs['IR_VJ_1_junction_aa'].astype(str) + '+' + adata.obs['IR_VDJ_1_junction_aa'].astype(str)
clono_dict = {clone: idx for idx, clone in enumerate(adata.obs['TRA+TRB'].unique())}
adata.obs['clonotype'] = adata.obs['TRA+TRB'].map(clono_dict)
print(len(adata.obs['clonotype'].unique()))
adata.obs['clonotype'].head()

### Embed Proteins

Append both chains, add start ('<'), stop ('>') and seperator ('+') token. For training purpose IR data might be needed as one hot vector.

In [ ]:
import sys
sys.path.append('../mvTCR')
import tcr_embedding.utils_preprocessing as utils

In [ ]:
adata.obs['TRA+TRB'] = adata.obs['IR_VJ_1_junction_aa'].astype(str) + '+' + adata.obs['IR_VDJ_1_junction_aa'].astype(str)
pad = adata.obs['TRA+TRB'].str.len().max()
pad = int(pad)
pad

In [ ]:
aa_to_id = {'_': 0, 'A': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10, 'M': 11, 'N': 12, 'P': 13,
            'Q': 14, 'R': 15, 'S': 16, 'T': 17, 'V': 18, 'W': 19, 'Y': 20, '+': 21, '<': 22, '>': 23}
utils.aa_encoding(adata, read_col='TRA+TRB', ohe_col='one_hot', label_col='tcr_seq', length_col='seq_len', pad=pad, aa_to_id=aa_to_id, start_end_symbol=True)


In [ ]:
print(adata.obsm['tcr_seq'].shape)
print(adata.obsm['one_hot'].shape)
print(adata.uns['aa_to_id'])

### Antigen Binding Encoding

Process the Binding Specificity to form that it easy to use during training.

In [ ]:
antigen_binding_list = [x for x in adata.obs.columns if x.endswith('_binder')]
binding_matrix = (adata.obs[antigen_binding_list] == 'True').values.astype(int)
binding_matrix.shape

In [ ]:
unique_value, count = np.unique(binding_matrix.sum(axis=1), return_counts=True)
for unique_value_, count_ in zip(unique_value, count):
    print(f'Cells with {unique_value_} bindings: {count_}')

In [ ]:
label_to_binding = {label_id: antigen for label_id, antigen in enumerate(antigen_binding_list)}
label_to_binding[-1] = 'no_data'  # For cells without binding data
binding_to_label = {antigen: label_id for label_id, antigen in enumerate(antigen_binding_list)}
binding_to_label['no_data'] = -1
binding_to_label == {v: k for k, v in label_to_binding.items()}  # sanity check, if both dicts are the reverse of each other

In [ ]:
adata.obs['has_binding'] = binding_matrix.sum(axis=1).astype(bool)
adata.obs['has_binding'].head()

In [ ]:
adata.obs['binding_label'] = np.argmax(binding_matrix, axis=1)
adata.obs['binding_label'][~adata.obs['has_binding']] = -1
adata.obs['binding_label'].head()

In [ ]:
adata.obs['binding_name'] = adata.obs['binding_label'].map(label_to_binding)
adata.obs['binding_name'].head()

### Test-Validation Split
We don't want to include the same clonotype in the different splits. Further we want to have the same data distribution in donor and antigen binding dimension to have faithful results.

Therefore we need stratification (similar label distribution across the sets) and grouped sampling (groups, i.e. the same clonotype is only present in one set)

In [ ]:
adata.obs['donor+binding'] = adata.obs['donor'].astype(str) + '_' + adata.obs['binding_name'].astype(str)
adata.obs['donor+binding'].sample(10)

In [ ]:
random_seed = 15
# Create Train-Val and Test set
train_val, test = utils.stratified_group_shuffle_split(adata.obs, stratify_col='donor+binding', group_col='clonotype', val_split=0.20, random_seed=random_seed)
# Split Train-Val into Train and Val set
train, val = utils.stratified_group_shuffle_split(train_val, stratify_col='donor+binding', group_col='clonotype', val_split=0.25, random_seed=random_seed)

In [ ]:
print('Train Samples: ', len(train))
print('Train Ratio: ', len(train) / len(adata.obs))
print('Val Samples: ', len(val))
print('Val Ratio: ', len(val) / len(adata.obs))
print('Test Samples: ', len(test))
print('Test Ratio: ', len(test) / len(adata.obs))

#### Visualize Distribution

Logarithmic scale

In [ ]:
pl.figure(figsize=(15,5))
pl.hist([train['binding_name'], val['binding_name'], test['binding_name']], density=True, bins=len(train['binding_name'].unique()), label=['train', 'val', 'test'])
pl.yscale('log')
pl.xticks(rotation='vertical')
pl.legend(loc='upper right')
pl.show()

Linear scale

In [ ]:
pl.figure(figsize=(15,5))
pl.hist([train['binding_name'], val['binding_name'], test['binding_name']], density=True, bins=len(train['binding_name'].unique()), label=['train', 'val', 'test'])
pl.xticks(rotation='vertical')
pl.legend(loc='upper right')
pl.show()

In [ ]:
pl.figure(figsize=(15,5))
pl.hist([train['donor'], val['donor'], test['donor']], density=True, bins=len(train['donor'].unique()), label=['train', 'val', 'test'])
pl.xticks(rotation='vertical')
pl.legend(loc='upper right')
pl.show()

#### Check if clonotypes are unique within each set

In [ ]:
form_train = set(train['clonotype'].tolist())
form_val = set(val['clonotype'].tolist())
form_test = set(test['clonotype'].tolist())

print('Intersection in clonotypes between two sets should be empty\n')
print('Intersection in clonotypes between train and test: ', form_train.intersection(form_test))
print('Intersection in clonotypes between train and val: ', form_train.intersection(form_val))
print('Intersection in clonotypes between val and test: ', form_val.intersection(form_test))
# print('\nSanity Check - Intersection in clonotypes between train and train: ', form_train.intersection(form_train))

#### Save resulting split

In [ ]:
adata.obs.loc[train.index, 'set'] = 'train'
adata.obs.loc[val.index, 'set'] = 'val'
adata.obs.loc[test.index, 'set'] = 'test'
adata.obs['set'].value_counts()

In [ ]:
adata.obs['set'].sample(20)

In [ ]:
adata.write_h5ad(path_clean_split, compression='gzip')

## Supervised and Semi-supervised preprocessing
Add prediction labels to adata

In [ ]:
import sys
sys.path.append('../mvTCR/')
import tcr_embedding as tcr

In [ ]:
adata = sc.read_h5ad(path_clean_split)

In [ ]:
adata.obs[['binding_name', 'binding_label']]

In [ ]:
import config.constants_10x as constants
high_count_antigens = constants.HIGH_COUNT_ANTIGENS.copy()
high_count_antigens

In [ ]:
adata.obs['high_count_binding_name'] = adata.obs['binding_name']
# Set rare antigen specificities to 'no_data'
adata.obs['high_count_binding_name'][~adata.obs['high_count_binding_name'].isin(high_count_antigens)] = 'no_data'
adata.obs['high_count_binding_name'] = adata.obs['high_count_binding_name'].astype(str)
adata.obs['high_count_binding_name'].unique()

In [ ]:
# Use this high_count_antigens list to be consistent with the order (alphabetical)
high_count_antigens += ['no_data']
specificity_to_label = {k: v for v, k in enumerate(high_count_antigens)}
label_to_specificity = {k: v for k, v in enumerate(high_count_antigens)}
adata.uns['specificity_to_label'] = specificity_to_label
adata.uns['label_to_specificity'] = label_to_specificity
specificity_to_label

In [ ]:
adata.obs['high_count_binding_label'] = adata.obs['high_count_binding_name'].map(specificity_to_label)
adata.obs['high_count_binding_label'].value_counts()

In [ ]:
aa_to_id = adata.uns['aa_to_id']
utils.aa_encoding(adata, read_col='IR_VJ_1_junction_aa', label_col='alpha_seq', length_col='alpha_len', pad=26, aa_to_id=aa_to_id, start_end_symbol=False)
utils.aa_encoding(adata, read_col='IR_VDJ_1_junction_aa', label_col='beta_seq', length_col='beta_len', pad=26, aa_to_id=aa_to_id, start_end_symbol=False)

In [ ]:
print(adata.uns.keys())
adata.uns = {}
adata.uns['aa_to_id'] = aa_to_id
adata.uns['specificity_to_label'] = specificity_to_label

In [ ]:
adata.write_h5ad(path_supervised, compression='gzip')

## Add avidity information

In [ ]:
import sys
sys.path.append('../mvTCR/')
import config.constants_10x as const

In [ ]:
adata = sc.read(path_supervised)

In [ ]:
cols_binder_counts = const.HIGH_COUNT_ANTIGENS
cols_binder_counts = ['_'.join(el.split('_')[:-1]) for el in cols_binder_counts]

In [ ]:
binding_counts = []
for donor in range(1, 5):
    path_binding = f'../mvTCR/data/10x_CD8TC/patient_{donor}/vdj_v1_hs_aggregated_donor{donor}_binarized_matrix.csv'
    binarized_matrix = pd.read_csv(path_binding, sep=',', header=0, index_col=0)
    binarized_matrix.index.name = None
    binarized_matrix = binarized_matrix[cols_binder_counts]
    binarized_matrix.index = binarized_matrix.index + f'-donor_{donor}'
    binding_counts.append(binarized_matrix)
binding_counts = pd.concat(binding_counts)

In [ ]:
binding_counts = binding_counts.loc[adata.obs.index]
adata.obsm['binding_counts'] = binding_counts.values

In [ ]:
adata.write_h5ad(path_avidity, compression='gzip')